In [2]:
from algorithm_ranking import MeasurementsManager
from backend_manager import BackendManager, Commands
import os

from am4pa.runners import RunnerVariants
from am4pa.data_integration import DataCollector
from am4pa.data_proccessing import CaseDurationsManager
from am4pa.data_proccessing import FilterOnKPIs

In [44]:
#should go into am4pa
class LinneaConfig:
    def __init__(self, problem, threads):
        
        self.problem = problem
        self.threads = threads
        #self.op_sizes = op_sizes
        #self.num_ops = len(op_sizes)
        
        self.local_dir = ""
        
        # Backend set up
        self.server = ""
        self.uname = ""
        self.server = ""
        self.backend_dir = None
        self.backend = False
        self.bm = None
        self.bm_cmds = None
        
        self.generation_script = 'generate-variants-linnea.py'
        self.measurements_script = "generate-measurements-script.py"
        self.local_bfolder = "cluster-data"
        self.slrum_submit_cmd = None
        
        self.app = "julia"
        self.runner_script = "runner.jl"
        self.runner_competing_script = 'runner_competing_{}.jl'
        
    def set_local_directory(self,local_dir):
        self.local_dir = local_dir
        
    def setup_backend_details(self, server, uname, init_script, backend_dir):
        self.server = server
        self.uname = uname
        self.init_script = init_script
        self.backend_dir = backend_dir
        self.backend = True
        self.slrum_submit_cmd = "sbatch submit.sh"
        
    def connect_backend(self):
        if self.backend:
            self.bm = BackendManager(server=self.server, uname = self.uname)
            self.bm.connect()
            self.bm_cmds = Commands(source = self.init_script)
        
            
    def sync_local_and_backend(self):
        if self.bm:
            if self.bm.connected:
                self.bm.copy_to_backend(self.local_dir, self.backend_dir)
            
    def check_backend_folder_sync(self):
        if self.bm:
            return self.bm.check_if_file_exists(os.path.join(self.backend_dir,self.generation_script))
        return False
        

In [45]:
class MeasurementsLinnea(MeasurementsManager):
    def __init__(self,linnea_config, op_sizes):
        super().__init__()
        
        self.linnea_config = linnea_config
        self.op_sizes = op_sizes
        self.runner = None
        self.data_collector = None
        self.case_durations_manager = None
        self.h0 = None
        
    def generate_variants(self):
        if self.linnea_config.backend:
            self.runner = RunnerVariants(self.op_sizes,
                                    self.linnea_config.backend_dir,
                                    threads = self.linnea_config.threads,
                                    backend_manager = self.linnea_config.bm,
                                    backend_commands= self.linnea_config.bm_cmds)
            
            local_operands_dir = os.path.join(self.linnea_config.local_dir,self.linnea_config.local_bfolder,self.runner.operands_dir_name)
            if not os.path.exists(local_operands_dir):
                os.makedirs(local_operands_dir)
            
            self.data_collector = DataCollector(local_operands_dir,
                                           self.runner.operands_dir, self.linnea_config.bm)
                
        else:
            self.runner = RunnerVariants(self.op_sizes,
                                    self.linnea_config.local_dir,
                                    threads = self.linnea_config.threads)
            self.data_collector = DataCollector(self.runner.operands_dir)
            
        self.runner.generate_variants_for_measurements(self.linnea_config.generation_script)
        

        
    def gather_competing_variants(self, bmeasure_once=False, rel_duration=1.2):
        case_table = self.data_collector.get_case_table()
        
        if bmeasure_once:
            self.runner.measure_variants(app=self.linnea_config.app,
                                        runner_script=self.linnea_config.runner_script)
            measurements_table = self.data_collector.get_runtimes_table()
            
            filterKPI = FilterOnKPIs(case_table,measurements_table)
            df = filterKPI.filter_on_flops_and_rel_duration(rel_duration)
            
            self.h0 = df['case:concept:name'].tolist()
        else:
            self.h0 = case_table['case:concept:name'].tolist()
            
        self.case_durations_manager = CaseDurationsManager()
        return self.h0
    
    def measure(self, rep_steps,run_id, bSlrum=False):
        self.runner.generate_measurements_script(self.linnea_config.measurements_script,
                                                self.h0,
                                                run_id,
                                                rep_steps)
        
        if not bSlrum:
            self.runner.measure_variants(app=self.linnea_config.app,
                                    runner_script=self.linnea_config.runner_competing_script.format(run_id))
            
            self.collect_measurements(run_id)
        else:
            self.runner.measure_variants(app=self.linnea_config.app,
                                    runner_script=self.linnea_config.runner_competing_script.format(run_id),
                                        submit_cmd=self.linnea_config.slrum_submit_cmd)
            
        
    def collect_measurements(self, run_id):
        df = self.data_collector.get_runtimes_competing_table(run_id)
        self.case_durations_manager.add_case_durations(df)
        
    def get_alg_measurements(self):
        return self.case_durations_manager.get_alg_measurements()
        

### Local

In [5]:
linnea_config = LinneaConfig("MatrixChain4",4)

In [6]:
linnea_config.set_local_directory("../linnea/matrix-chain-4/")

In [7]:
op_sizes = ["100","90","6","90","100"]
ms = MeasurementsLinnea(linnea_config,op_sizes)

In [8]:
ms.generate_variants()

['python', '../linnea/matrix-chain-4/generate-variants-linnea.py', '100', '90', '6', '90', '100', '--threads=4']
New solution:.............3.24e+05
No further generation steps possible.
----------------------------------
Number of nodes:                 8
Solution nodes:                  1
Data:                     2.81e+04
Best solution:            3.24e+05
Intensity:                    11.5
Number of algorithms:            6
Generated Variants.
Success: Local run: Generate variants


In [9]:
ms.gather_competing_variants()

['algorithm1',
 'algorithm5',
 'algorithm4',
 'algorithm0',
 'algorithm3',
 'algorithm2']

In [14]:
ms.measure(3,2)

['python', '../linnea/matrix-chain-4/experiments/100_90_6_90_100/generate-measurements-script.py', '--algs', 'algorithm1', 'algorithm5', 'algorithm4', 'algorithm0', 'algorithm3', 'algorithm2', '--rep', '3', '--threads', '4', '--id', '2']
Success: Local run: Generate Measurement script 2
Running Measurements locally
['julia', '../linnea/matrix-chain-4/experiments/100_90_6_90_100/runner_competing_2.jl']
Success: Local run: Measurements from runner_competing_2.jl


In [15]:
ms.get_alg_measurements()

{'algorithm4': [0.00015497207641601562,
  0.00019288063049316406,
  0.00029087066650390625,
  0.00015687942504882812,
  0.00016307830810546875,
  0.00017404556274414062,
  0.00015497207641601562,
  0.0001690387725830078,
  0.0002238750457763672],
 'algorithm1': [6.890296936035156e-05,
  6.985664367675781e-05,
  6.890296936035156e-05,
  5.91278076171875e-05,
  5.793571472167969e-05,
  7.796287536621094e-05,
  5.698204040527344e-05,
  5.817413330078125e-05,
  5.817413330078125e-05],
 'algorithm3': [0.00011801719665527344,
  0.00011801719665527344,
  0.00019884109497070312,
  0.00012993812561035156,
  0.00013303756713867188,
  0.0001239776611328125,
  0.0001289844512939453,
  0.0001220703125,
  0.0001239776611328125],
 'algorithm0': [7.081031799316406e-05,
  6.103515625e-05,
  5.4836273193359375e-05,
  6.604194641113281e-05,
  5.793571472167969e-05,
  5.3882598876953125e-05,
  6.103515625e-05,
  5.5789947509765625e-05,
  5.91278076171875e-05],
 'algorithm2': [0.0008649826049804688,
  0.00

### Backend

In [58]:
linnea_config = LinneaConfig("MatrixChain4",4)

In [59]:
linnea_config.set_local_directory("../linnea/matrix-chain-4/")
linnea_config.setup_backend_details(server="login18-1.hpc.itc.rwth-aachen.de",
                                   uname = "as641651",
                                   init_script="~/.analyzer",
                                   backend_dir="~/PhD/va4algs-exps/matrix-chain-4/")

In [60]:
linnea_config.connect_backend()

In [61]:
linnea_config.check_backend_folder_sync()

True

In [62]:
op_sizes = ["100","90","6","90","100"]
ms = MeasurementsLinnea(linnea_config,op_sizes)

In [63]:
ms.generate_variants()

source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4; python generate-variants-linnea.py 100 90 6 90 100 --threads=4
['New solution:.............3.24e+05\n', 'No further generation steps possible.\n', '----------------------------------\n', 'Number of nodes:                 8\n', 'Solution nodes:                  1\n', 'Data:                     2.81e+04\n', 'Best solution:            3.24e+05\n', 'Intensity:                    11.5\n', 'Number of algorithms:            6\n', 'Generated Variants.\n']
Success: Backend interactive run: Generate variants


In [64]:
ms.gather_competing_variants()

['algorithm0',
 'algorithm2',
 'algorithm4',
 'algorithm1',
 'algorithm5',
 'algorithm3']

In [41]:
ms.measure(3,1)

source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4/experiments/100_90_6_90_100; python generate-measurements-script.py --algs algorithm0 algorithm2 algorithm4 algorithm1 algorithm5 algorithm3 --rep 3 --threads 4 --id 1
[]
Success: Backend interactive run: Generate Measurement script 1
Running Measurements Backend interactive
source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4/experiments/100_90_6_90_100; julia runner_competing_1.jl 
[]
Success: Backend interactive run: Measurements from runner_competing_1.jl


In [42]:
ms.get_alg_measurements()

{'algorithm4': [0.000209808349609375,
  0.00017905235290527344,
  0.0001990795135498047],
 'algorithm3': [0.001756906509399414,
  0.00015091896057128906,
  0.000141143798828125],
 'algorithm1': [6.699562072753906e-05,
  7.104873657226562e-05,
  6.508827209472656e-05],
 'algorithm0': [7.605552673339844e-05,
  6.914138793945312e-05,
  7.009506225585938e-05],
 'algorithm2': [0.00012612342834472656,
  0.0001399517059326172,
  0.00012993812561035156],
 'algorithm5': [0.00020384788513183594,
  0.00021791458129882812,
  0.00021004676818847656]}

### SLRUM Measurement

In [70]:
ms.measure(3,5,bSlrum=True)

source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4/experiments/100_90_6_90_100; python generate-measurements-script.py --algs algorithm0 algorithm2 algorithm4 algorithm1 algorithm5 algorithm3 --rep 3 --threads 4 --id 5
[]
Success: Backend interactive run: Generate Measurement script 5
Running Measurements Backend batch
source ~/.analyzer; cd ~/PhD/va4algs-exps/matrix-chain-4/experiments/100_90_6_90_100; sbatch submit.sh julia 'runner_competing_5.jl '
['Submitted batch job 31421523\n']
Success: Backend batch run: Measurements from runner_competing_5.jl


In [74]:
linnea_config.bm.check_slrum_status(ms.runner.job_name)

['             JOBID PARTITION                           NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)\n']


0

In [75]:
ms.collect_measurements(5)

scp as641651@login18-1.hpc.itc.rwth-aachen.de:~/PhD/va4algs-exps/matrix-chain-4/experiments/100_90_6_90_100/run_times_competing_5.csv ../linnea/matrix-chain-4/cluster-data/100_90_6_90_100
b''


In [76]:
ms.get_alg_measurements()

{'algorithm4': [0.000102996826171875,
  0.00011897087097167969,
  0.00011682510375976562,
  0.00012922286987304688,
  0.0001380443572998047,
  0.00012087821960449219],
 'algorithm3': [0.00016117095947265625,
  9.298324584960938e-05,
  9.012222290039062e-05,
  0.00010609626770019531,
  9.107589721679688e-05,
  8.988380432128906e-05],
 'algorithm1': [5.0067901611328125e-05,
  4.506111145019531e-05,
  4.38690185546875e-05,
  3.0994415283203125e-05,
  4.1961669921875e-05,
  4.315376281738281e-05],
 'algorithm0': [4.601478576660156e-05,
  4.100799560546875e-05,
  4.1961669921875e-05,
  4.38690185546875e-05,
  4.9114227294921875e-05,
  4.291534423828125e-05],
 'algorithm2': [5.698204040527344e-05,
  6.103515625e-05,
  7.891654968261719e-05,
  8.20159912109375e-05,
  8.797645568847656e-05,
  8.20159912109375e-05],
 'algorithm5': [0.000125885009765625,
  0.00012302398681640625,
  0.000125885009765625,
  0.00012302398681640625,
  0.00013017654418945312,
  0.00012493133544921875]}